In [ ]:
# Import necessary libraries
import pandas as pd
import PyPDF2
import spacy
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import gradio as gr

In [ ]:
# Define functions for text extraction and embedding
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

In [ ]:
 #Path to your PDF file
#pdf_path = '/content/Getting Started with Ubuntu 16.04.pdf'  # Adjust path as needed


In [ ]:
# Extract text from the PDF
#pdf_text = extract_text_from_pdf(pdf_path)

# Convert the text to a DataFrame
#df = pd.DataFrame({'text': [pdf_text]})

In [ ]:
df1=pd.read_csv('/content/dialogueText_file1.csv')


In [ ]:
# prompt: store 30 percent of df1 in df

df = df1.sample(frac=0.3)


In [ ]:
# Load the custom embedding model
class CustomEmbeddingModel:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def embed_text(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            embeddings = self.model(**inputs).last_hidden_state.mean(dim=1)
        return embeddings[0].numpy()

embedding_model = CustomEmbeddingModel('/content/Embedding_model')  # Replace with your model name


In [ ]:
# Load Spacy model for preprocessing
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if token.text.lower() not in spacy.lang.en.stop_words.STOP_WORDS and token.is_alpha]
    return ' '.join(tokens)


In [ ]:
# Apply preprocessing and embedding
df['text'] = df['text'].apply(preprocess_text)
df['text_embeddings'] = df['text'].apply(lambda x: embedding_model.embed_text(x))

In [ ]:
# Create FAISS vector store
documents = df['text'].tolist()
embeddings = df['text_embeddings'].tolist()
index = faiss.IndexFlatL2(len(embeddings[0]))
index.add(np.array(embeddings))

In [ ]:
# Function to perform similarity search
def search(query):
    query_embedding = embedding_model.embed_text(query).reshape(1, -1)
    _, indices = index.search(query_embedding, k=1)  # Retrieve top 1 result
    return documents[indices[0][0]]


In [ ]:
# Function to generate a response
def generate_response(prompt):
    result = search(prompt)
    return result

In [ ]:
# Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(label="Enter your query", placeholder="Ask about Ubuntu..."),
    outputs=gr.Textbox(label="Response"),
    title="Ubuntu Manual Chatbot",
    description="Ask questions about the Ubuntu manual."
)

In [ ]:
# Launch the Gradio interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bc0dcd839f0c2fc18b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
